<a href="https://colab.research.google.com/github/tgwon/ai4ml/blob/main/project/modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 라이브러리 import 및 설치

In [1]:
import os
import random
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.0 MB/s eta 0:00:00


In [64]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.6 MB/s eta 0:00:00


In [82]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 12.3 MB/s eta 0:00:00


## 데이터 준비 및 라벨 인코딩

In [29]:
train_org = pd.read_csv('/content/drive/MyDrive/ai4ml/open/final_train_df.csv')
test_org = pd.read_csv('/content/drive/MyDrive/ai4ml/open/final_test_df.csv')

train_df = train_org.copy()
test_df = test_org.copy()

In [32]:
train_df.head()

,ID,요일,기상상태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO,outlier,시군구별평균ECLO,시군구별사고발생횟수,구,동,연,월,일,시간,special_month,활동시간대,holiday,도로형태1,도로형태2,계절,설치개수,어린이보호구역점수,급지구분_1,급지구분_2,급지구분_3,cctv점수
0,ACCIDENT_00000,화요일,맑음,건조,차대사람,길가장자리구역통행중,안전운전불이행,승용,여,51세,상해없음,보행자,여,70세,중상,0,1,0,0,5,False,4.282443,131,중구,대신동,2019,1,1,0,1,0,1,단일로,기타,겨울,391.0,2.0,11.0,0.0,0.0,3.0
1,ACCIDENT_00001,화요일,기타,건조,차대사람,보도통행중,기타,승용,남,39세,상해없음,보행자,남,61세,경상,0,0,1,0,3,False,4.738938,678,달서구,감삼동,2019,1,1,0,1,0,1,단일로,기타,겨울,932.0,0.0,0.0,8.0,3.0,8.0
2,ACCIDENT_00002,화요일,맑음,건조,차대사람,차도통행중,안전운전불이행,승용,남,70세,상해없음,보행자,남,38세,경상,0,0,1,0,3,False,4.842715,604,수성구,두산동,2019,1,1,1,1,0,1,단일로,기타,겨울,473.0,5.0,0.0,0.0,0.0,4.0
3,ACCIDENT_00003,화요일,맑음,건조,차대차,추돌,안전운전불이행,승용,남,49세,상해없음,승용,남,36세,중상,0,1,0,0,5,False,4.208920,426,북구,복현동,2019,1,1,2,1,0,1,단일로,기타,겨울,534.0,10.0,0.0,10.0,5.0,10.0
4,ACCIDENT_00004,화요일,맑음,건조,차대차,추돌,안전운전불이행,승용,남,30세,상해없음,승용,남,52세,경상,0,0,1,0,3,False,4.549091,825,동구,신암동,2019,1,1,4,1,0,1,단일로,기타,겨울,2057.0,15.0,1.0,4.0,0.0,12.0


In [33]:
test_df.head()

,ID,요일,기상상태,노면상태,사고유형,시군구별평균ECLO,시군구별사고발생횟수,구,동,연,월,일,시간,special_month,활동시간대,holiday,도로형태1,도로형태2,계절,설치개수,어린이보호구역점수,급지구분_1,급지구분_2,급지구분_3,cctv점수
0,ACCIDENT_39609,토요일,맑음,건조,차대사람,4.881657,338,수성구,상동,2022,1,1,1,1,0,1,교차로,교차로안,겨울,700.0,5.0,0.0,0.0,0.0,4.0
1,ACCIDENT_39610,토요일,맑음,건조,차대사람,4.563008,492,수성구,지산동,2022,1,1,1,1,0,1,단일로,기타,겨울,0.0,10.0,0.0,0.0,2.0,7.0
2,ACCIDENT_39611,토요일,맑음,건조,차대차,4.945578,147,수성구,수성동2가,2022,1,1,4,1,0,1,교차로,교차로안,겨울,0.0,1.0,0.0,0.0,0.0,1.0
3,ACCIDENT_39612,토요일,맑음,건조,차대차,4.438172,372,수성구,신매동,2022,1,1,4,1,0,1,단일로,기타,겨울,0.0,7.0,0.0,2.0,1.0,11.0
4,ACCIDENT_39613,토요일,맑음,건조,차대차,4.738938,678,달서구,감삼동,2022,1,1,6,1,1,1,교차로,교차로안,겨울,932.0,0.0,0.0,8.0,3.0,8.0


In [34]:
train_df.columns

Index(['ID', '요일', '기상상태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반', '가해운전자 차종',
       '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별',
       '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수', 'ECLO',
       'outlier', '시군구별평균ECLO', '시군구별사고발생횟수', '구', '동', '연', '월', '일', '시간',
       'special_month', '활동시간대', 'holiday', '도로형태1', '도로형태2', '계절', '설치개수',
       '어린이보호구역점수', '급지구분_1', '급지구분_2', '급지구분_3', 'cctv점수'],
      dtype='object')

In [35]:
test_df.columns

Index(['ID', '요일', '기상상태', '노면상태', '사고유형', '시군구별평균ECLO', '시군구별사고발생횟수', '구',
       '동', '연', '월', '일', '시간', 'special_month', '활동시간대', 'holiday', '도로형태1',
       '도로형태2', '계절', '설치개수', '어린이보호구역점수', '급지구분_1', '급지구분_2', '급지구분_3',
       'cctv점수'],
      dtype='object')

In [36]:
# '연', '월', '일', '시간' 열의 dtype을 object로 변경
train_df['연'] = train_df['연'].astype(str)
train_df['월'] = train_df['월'].astype(str)
train_df['일'] = train_df['일'].astype(str)
train_df['시간'] = train_df['시간'].astype(str)

In [37]:
# '연', '월', '일', '시간' 열의 dtype을 object로 변경
test_df['연'] = test_df['연'].astype(str)
test_df['월'] = test_df['월'].astype(str)
test_df['일'] = test_df['일'].astype(str)
test_df['시간'] = test_df['시간'].astype(str)

In [41]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39573 entries, 0 to 39572
Data columns (total 41 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             39573 non-null  object 
 1   요일             39573 non-null  object 
 2   기상상태           39573 non-null  object 
 3   노면상태           39573 non-null  object 
 4   사고유형           39573 non-null  object 
 5   사고유형 - 세부분류    39573 non-null  object 
 6   법규위반           39573 non-null  object 
 7   가해운전자 차종       39573 non-null  object 
 8   가해운전자 성별       39573 non-null  object 
 9   가해운전자 연령       39573 non-null  object 
 10  가해운전자 상해정도     39573 non-null  object 
 11  피해운전자 차종       39573 non-null  object 
 12  피해운전자 성별       39573 non-null  object 
 13  피해운전자 연령       39573 non-null  object 
 14  피해운전자 상해정도     39573 non-null  object 
 15  사망자수           39573 non-null  int64  
 16  중상자수           39573 non-null  int64  
 17  경상자수           39573 non-null  int64  
 18  부상자수  

In [39]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10963 entries, 0 to 10962
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             10963 non-null  object 
 1   요일             10963 non-null  object 
 2   기상상태           10963 non-null  object 
 3   노면상태           10963 non-null  object 
 4   사고유형           10963 non-null  object 
 5   시군구별평균ECLO     10963 non-null  float64
 6   시군구별사고발생횟수     10963 non-null  int64  
 7   구              10963 non-null  object 
 8   동              10963 non-null  object 
 9   연              10963 non-null  object 
 10  월              10963 non-null  object 
 11  일              10963 non-null  object 
 12  시간             10963 non-null  object 
 13  special_month  10963 non-null  int64  
 14  활동시간대          10963 non-null  int64  
 15  holiday        10963 non-null  int64  
 16  도로형태1          10963 non-null  object 
 17  도로형태2          10963 non-null  object 
 18  계절    

In [42]:
test_x = test_df.drop(columns=['ID']).copy()

In [43]:
eclo = train_df['ECLO']

In [44]:
train_df = train_df[test_x.columns].copy()

In [45]:
train_df['ECLO'] = eclo

In [46]:
train_df.head()

,요일,기상상태,노면상태,사고유형,시군구별평균ECLO,시군구별사고발생횟수,구,동,연,월,일,시간,special_month,활동시간대,holiday,도로형태1,도로형태2,계절,설치개수,어린이보호구역점수,급지구분_1,급지구분_2,급지구분_3,cctv점수,ECLO
0,화요일,맑음,건조,차대사람,4.282443,131,중구,대신동,2019,1,1,0,1,0,1,단일로,기타,겨울,391.0,2.0,11.0,0.0,0.0,3.0,5
1,화요일,기타,건조,차대사람,4.738938,678,달서구,감삼동,2019,1,1,0,1,0,1,단일로,기타,겨울,932.0,0.0,0.0,8.0,3.0,8.0,3
2,화요일,맑음,건조,차대사람,4.842715,604,수성구,두산동,2019,1,1,1,1,0,1,단일로,기타,겨울,473.0,5.0,0.0,0.0,0.0,4.0,3
3,화요일,맑음,건조,차대차,4.208920,426,북구,복현동,2019,1,1,2,1,0,1,단일로,기타,겨울,534.0,10.0,0.0,10.0,5.0,10.0,5
4,화요일,맑음,건조,차대차,4.549091,825,동구,신암동,2019,1,1,4,1,0,1,단일로,기타,겨울,2057.0,15.0,1.0,4.0,0.0,12.0,3


In [47]:
# from sklearn.preprocessing import LabelEncoder

# labelencoder가 아니라 targerencoder 사용
from category_encoders.target_encoder import TargetEncoder

categorical_features = list(train_df.dtypes[train_df.dtypes == "object"].index)

for i in categorical_features:
    le = TargetEncoder(cols=[i])
    train_df[i] = le.fit_transform(train_df[i], train_df['ECLO'])

# 추출된 문자열 변수 확인
print(categorical_features)

['요일', '기상상태', '노면상태', '사고유형', '구', '동', '연', '월', '일', '시간', '도로형태1', '도로형태2', '계절']


In [48]:
train_df.head()

,요일,기상상태,노면상태,사고유형,시군구별평균ECLO,시군구별사고발생횟수,구,동,연,월,일,시간,special_month,활동시간대,holiday,도로형태1,도로형태2,계절,설치개수,어린이보호구역점수,급지구분_1,급지구분_2,급지구분_3,cctv점수,ECLO
0,4.626747,4.711899,4.711895,3.816903,4.282443,131,4.541971,4.282449,4.840196,4.662153,4.816007,5.071256,1,0,1,4.669835,4.597547,4.655799,391.0,2.0,11.0,0.0,0.0,3.0,5
1,4.626747,4.789407,4.711895,3.816903,4.738938,678,4.617796,4.738938,4.840196,4.662153,4.816007,5.071256,1,0,1,4.669835,4.597547,4.655799,932.0,0.0,0.0,8.0,3.0,8.0,3
2,4.626747,4.711899,4.711895,3.816903,4.842715,604,4.727110,4.842715,4.840196,4.662153,4.816007,5.251121,1,0,1,4.669835,4.597547,4.655799,473.0,5.0,0.0,0.0,0.0,4.0,3
3,4.626747,4.711899,4.711895,4.943519,4.208920,426,4.687227,4.208920,4.840196,4.662153,4.816007,5.412331,1,0,1,4.669835,4.597547,4.655799,534.0,10.0,0.0,10.0,5.0,10.0,5
4,4.626747,4.711899,4.711895,4.943519,4.549091,825,4.884023,4.549091,4.840196,4.662153,4.816007,5.139726,1,0,1,4.669835,4.597547,4.655799,2057.0,15.0,1.0,4.0,0.0,12.0,3


## 모델링

In [88]:
from sklearn.model_selection import train_test_split

X = train_df.drop('ECLO', axis=1)
y = train_df['ECLO']

# 학습(train), 검증(validation), 시험(test) 세트로 나누기
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [86]:
from sklearn.metrics import mean_squared_log_error
def rmsle(y_true,y_pred):
    return mean_squared_log_error(y_true,y_pred) ** 0.5

In [96]:
import optuna
import xgboost as xgb
from optuna import Trial, visualization
from optuna.samplers import TPESampler

In [99]:
def objectiveXGB(trial: Trial, X, y, test):
    param = {
        "n_estimators" : trial.suggest_int('n_estimators', 500, 4000),
        'max_depth':trial.suggest_int('max_depth', 8, 16),
        'min_child_weight':trial.suggest_int('min_child_weight', 1, 300),
        'gamma':trial.suggest_int('gamma', 1, 3),
        'learning_rate': 0.01,
        'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.5, 1, 0.1),
        'nthread' : -1,
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'subsample': trial.suggest_categorical('subsample', [0.6,0.7,0.8,1.0] ),
        'random_state': 42
    }
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


    model = xgb.XGBRegressor(**param)
    xgb_model = model.fit(X_train, y_train, verbose=False, eval_set=[(X_test, y_test)])
    score = rmsle(xgb_model.predict(X_test), y_test)

    return score

In [100]:
def train(model):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

    model = model.fit(X_train, y_train, early_stopping_rounds=100, verbose=False, eval_set=[(X_test, y_test)])
    score = rmsle(model.predict(X_train), y_train)
    print(score)
    return model

In [101]:
study = optuna.create_study(direction='minimize',sampler=TPESampler())
study.optimize(lambda trial : objectiveXGB(trial, X,  y, X_test), n_trials=50)
print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))

best_param = study.best_trial.params
xgbReg = train(xgb.XGBRegressor(**best_param, tree_method='gpu_hist', random_state=42, predictor='gpu_predictor', learning_rate=0.01, nthread=-1))

# score : 0.44280497666105884

[I 2023-12-12 18:46:58,695] A new study created in memory with name: no-name-edb5a999-e376-46ae-972f-12850cc37c06
[I 2023-12-12 18:47:03,614] Trial 0 finished with value: 0.4580729954172667 and parameters: {'n_estimators': 1281, 'max_depth': 11, 'min_child_weight': 153, 'gamma': 2, 'colsample_bytree': 0.5, 'lambda': 2.1216910787376433, 'alpha': 0.4624413393988957, 'subsample': 1.0}. Best is trial 0 with value: 0.4580729954172667.
[I 2023-12-12 18:47:13,024] Trial 1 finished with value: 0.46778045219934045 and parameters: {'n_estimators': 703, 'max_depth': 15, 'min_child_weight': 23, 'gamma': 2, 'colsample_bytree': 0.9, 'lambda': 0.0013074828043868182, 'alpha': 0.013133894866960427, 'subsample': 0.7}. Best is trial 0 with value: 0.4580729954172667.
[I 2023-12-12 18:47:36,335] Trial 2 finished with value: 0.4724290107253115 and parameters: {'n_estimators': 3990, 'max_depth': 15, 'min_child_weight': 200, 'gamma': 2, 'colsample_bytree': 1.0, 'lambda': 0.0013012244005635727, 'alpha': 0.1159

Best trial: score 0.4560223092787301,
params {'n_estimators': 3880, 'max_depth': 8, 'min_child_weight': 51, 'gamma': 3, 'colsample_bytree': 1.0, 'lambda': 0.0010711029045690132, 'alpha': 0.21193290674125848, 'subsample': 1.0}
0.44280497666105884


In [102]:
optuna.visualization.plot_optimization_history(study)

In [103]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)

In [104]:
# 파라미터들관의 관계
optuna.visualization.plot_parallel_coordinate(study)

In [108]:
# 각 파라미터들의 상관관계
optuna.visualization.plot_contour(
    study,
    params=[
        "n_estimators",
        'max_depth',
        'min_child_weight',
        'gamma',
        'colsample_bytree',
        'lambda',
        'alpha',
        'subsample'
    ],
)